In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169
import tensorflow as tf
from skimage.transform import AffineTransform, warp

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics
import tensorflow_hub as hub

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [5]:
y = y.set_index("image_id")

In [6]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [7]:
def affine_image(img):
    """

    Args:
        img: (h, w) or (1, h, w)

    Returns:
        img: (h, w)
    """
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return (transformed_image*255).astype(np.uint8)

In [8]:
size=(224,224)

def calcRotate(img):
    detector = cv2.ORB_create()
    keypoints=detector.detect(img)
    descriptors=detector.compute(img,keypoints)
    angles=[]
    weights=[]
    for i in descriptors[0]:
        if i.angle!=-1:
            angles.append(i.angle)
            weights.append(i.response)
    if len(angles)==0:
        return 0
    else:
        return np.average(angles,weights=weights)

def upper(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[up:mid,:])==0:
            up=mid
        else:
            bottom=mid
    return max(up-mergin,0)

def lower(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[mid:bottom,:])==0:
            bottom=mid
        else:
            up=mid
    return min(bottom+mergin,img.shape[0])

def lefter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,left:mid])==0:
            left=mid
        else:
            right=mid
    return max(left-mergin,0)

def righter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,mid:right])==0:
            right=mid
        else:
            left=mid
    return min(right+mergin,img.shape[1])

def transformImg(img,size=(255,255),training=True):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if training:
        img=affine_image(img)
    img = img[upper(img):lower(img),lefter(img):righter(img)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img

def rotateImg(img, angle=None,prob=True):
    if prob==False or np.random.random()<0.5:
        if angle==None:
            angle=np.random.random()*60-30
        trans = cv2.getRotationMatrix2D((img.shape[0]/2, img.shape[1]/2), angle , 1.0)
        img = cv2.warpAffine(img, trans, (img.shape[1],img.shape[0]))
        return img
    else:
        return img

def shiftImg(img, prob=True):
    if prob==False or np.random.random()<0.5:
        x=np.random.random()*30-15
        y=np.random.random()*30-15
        trans = np.float32([[1,0,x],[0,1,y]])
        img = cv2.warpAffine(img, trans, (img.shape[1],img.shape[0]))
        return img
    else:
        return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [9]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(255,255)):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
            
        
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,training=self.training))
         
        
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training and np.random.random()<0.4:
            r = np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            alpha=np.random.random(imgs.shape[0])
            alpha[alpha>1]=1
            alpha[alpha<0]=0
            imgs=np.tile(alpha,(3,*size,1)).T*imgs+np.tile((1-alpha),(3,*size,1)).T*imgs2
            grapheme=np.tile(alpha,(168,1)).T*grapheme+np.tile((1-alpha),(168,1)).T*grapheme2
            vowel=np.tile(alpha,(11,1)).T*vowel+np.tile((1-alpha),(11,1)).T*vowel2
            consonant=np.tile(alpha,(7,1)).T*consonant+np.tile((1-alpha),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
        
            
            
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [10]:
"""
def getMultiBased():
    model =  ResNet152V2(weights="imagenet", include_top=False)
    x = model.output  
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(1024, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model
    """

def getMultiBased():
    feature_extractor_url = "https://tfhub.dev/google/efficientnet/b0/feature-vector/1"
    feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(*size,3))
    model=Sequential([feature_extractor_layer])
    x=model.output
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236),y,train_size=0.9,random_state=8000)
del df

In [12]:
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="tmp-eff-epoch{epoch:04}.h5")

In [ ]:
train_gen = DataLoader(X_train, y_train, training=True, batch_size=64,size=size)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=64,size=size)
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
#with strategy.scope():
model = getMultiBased()
model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[0.8,0.1,0.1])
print("compiled")
model.fit(train_gen, validation_data=valid_gen, epochs=80, callbacks=[checkpoint])
model.save("multiEfficient.h5")

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


compiled
Train for 2825 steps, validate for 314 steps
Epoch 1/80
2825/2825 [==============================] - 723s 256ms/step - loss: 3.0870 - dense_loss: 3.5989 - dense_1_loss: 1.2224 - dense_2_loss: 0.8555 - dense_acc: 0.2013 - dense_1_acc: 0.6032 - dense_2_acc: 0.6963 - val_loss: 2.4389 - val_dense_loss: 2.8705 - val_dense_1_loss: 0.7799 - val_dense_2_loss: 0.6450 - val_dense_acc: 0.3184 - val_dense_1_acc: 0.7458 - val_dense_2_acc: 0.7643
Epoch 2/80
2825/2825 [==============================] - 721s 255ms/step - loss: 2.6448 - dense_loss: 3.0805 - dense_1_loss: 1.0457 - dense_2_loss: 0.7581 - dense_acc: 0.2946 - dense_1_acc: 0.6691 - dense_2_acc: 0.7338 - val_loss: 2.1942 - val_dense_loss: 2.5807 - val_dense_1_loss: 0.7049 - val_dense_2_loss: 0.5912 - val_dense_acc: 0.3690 - val_dense_1_acc: 0.7694 - val_dense_2_acc: 0.7976
Epoch 3/80
2825/2825 [==============================] - 734s 260ms/step - loss: 2.5112 - dense_loss: 2.9203 - dense_1_loss: 1.0130 - dense_2_loss: 0.7364 - dense_